<a href="https://colab.research.google.com/github/mentalMint/coreference-resolution/blob/main/RuCoCo_baseline_joker_iis_nsk_su.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies

In [ ]:
!pip install -U pip setuptools wheel

  Using cached setuptools-69.2.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached wheel-0.43.0-py3-none-any.whl.metadata (2.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 4.6 MB/s eta 0:00:00
Using cached setuptools-69.2.0-py3-none-any.whl (821 kB)
Using cached wheel-0.43.0-py3-none-any.whl (65 kB)
  Attempting uninstall: wheel
    Found existing installation: wheel 0.41.2
    Uninstalling wheel-0.41.2:
      Successfully uninstalled wheel-0.41.2
  Attempting uninstall: setuptools
    Found existing installation: setuptools 68.2.2
    Uninstalling setuptools-68.2.2:
      Successfully uninstalled setuptools-68.2.2
  Attempting uninstall: pip
    Found existing installation: pip 23.3.1
    Uninstalling pip-23.3.1:
      Successfully uninstalled pip-23.3.1


In [ ]:
!python -m pip install --verbose transformers==4.15.0
# !python -m pip install --verbose transformers

Using pip 24.0 from /home/shuvalov/.conda/envs/env/lib/python3.9/site-packages/pip (python 3.9)
  Obtaining dependency information for transformers==4.15.0 from https://files.pythonhosted.org/packages/4a/7f/f1c28621af0d74794b18cbe5534ec7565ee782ba48257d08ec264bc4aacb/transformers-4.15.0-py3-none-any.whl.metadata
  Using cached transformers-4.15.0-py3-none-any.whl.metadata (59 kB)
  Obtaining dependency information for tokenizers<0.11,>=0.10.1 from https://files.pythonhosted.org/packages/a8/4f/ca8bc50358c3aaf50f298860a5ce1822e0c0ff97543e32265d1353760555/tokenizers-0.10.3-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata
  Using cached tokenizers-0.10.3-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (5.8 kB)
Using cached transformers-4.15.0-py3-none-any.whl (3.4 MB)
Using cached tokenizers-0.10.3-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2

In [ ]:
!pip install -U torch==1.13.1 --no-cache-dir

In [ ]:
!pip install -U spacy==3.2.1

In [ ]:
!python -m pip install pytorch-lightning==1.9.0

In [ ]:
!python -m spacy download ru_core_news_md

DEPRECATION: https://github.com/explosion/spacy-models/releases/download/ru_core_news_md-3.2.0/ru_core_news_md-3.2.0-py3-none-any.whl#egg=ru_core_news_md==3.2.0 contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 MB 351.3 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')


In [ ]:
!pip install -U huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.5/388.5 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.21.4
    Uninstalling huggingface-hub-0.21.4:
      Successfully uninstalled huggingface-hub-0.21.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
trl 0.4.7 requires transformers>=4.18.0, but you have transformers 4.15.0 which is incompatible.


In [ ]:
!pip install -U 'torch_xla'

In [ ]:
!pip install --no-cache-dir pickle5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp39-cp39-linux_x86_64.whl size=124786 sha256=5a1ad39c6520552bfca06d7246d136fad8b5a982375791beba703de3b2960921
  Stored in directory: /tmp/pip-ephem-wheel-cache-2tqffs24/wheels/f2/7a/49/9bef8878949914ecb90c08fc5bf30a05e17f475fe7e08b63a8
Successfully built pickle5


In [ ]:
pip install typing-inspect==0.8.0 typing_extensions==4.5.0

Note: you may need to restart the kernel to use updated packages.


# Used classes and methods

In [ ]:
import logging
from typing import Dict, Iterator, List, Optional, Tuple, List, NamedTuple
import transformers
import spacy
from tokenizers import pre_tokenizers
import pytorch_lightning as pl
import torch
from transformers import AutoModel
import json
from collections import Counter

# global_encoder_model_name = "cointegrated/rubert-tiny"
# global_encoder_model_name = "cointegrated/rubert-tiny2"
# global_encoder_model_name = "blinoff/roberta-base-russian-v0"
global_encoder_model_name = "ai-forever/ruRoberta-large"
# global_encoder_model_name = "kazzand/ru-longformer-tiny-16384"
# global_encoder_model_name = "kazzand/ru-longformer-large-4096"


max_text_length = 512

class SpanExtractor:
    skipped_pos = {"ADP", "CCONJ", "SCONJ"}
    allowed_punct = {"\"", "'", "(", ")", "."}

    def __init__(self, model_name: str = "ru_core_news_md"):
        self.nlp = spacy.load(model_name)

    def __call__(self, text: str) -> Iterator[Tuple[int, int]]:
        res = self.nlp(text, disable=("lemmatizer", "ner"))
        for token in res:
            if token.pos_ in {"DET", "PRON"}:             # определитель, местоимение
                yield token.idx, token.idx + len(token)
            elif token.pos_ in {"NOUN", "PROPN"}:
                start, end = token.idx, token.idx + len(token)
                for i in range(token.i - 1, -1, -1):
                    if (token.sent != res[i].sent
                            or not token.is_ancestor(res[i])):
                        break
                    if res[i].head == token:
                        if self._is_participle_phrase(res[i]):
                            break
                        if self._is_skipped_pos(res[i]):
                            continue
                        leftmost = next(node for node in res[i].subtree if not self._is_skipped_pos(node))
                        if leftmost.idx < start:
                            start = leftmost.idx
                for i in range(token.i + 1, len(res)):
                    if (token.sent != res[i].sent
                            or not token.is_ancestor(res[i])):
                        break
                    if res[i].head == token:
                        if self._is_participle_phrase(res[i]):
                            break
                        if self._is_skipped_pos(res[i]):
                            continue
                        rightmost = next(node for node in reversed(list(res[i].subtree))
                                         if not self._is_skipped_pos(node))
                        if rightmost.idx + len(rightmost) > end:
                            end = rightmost.idx + len(rightmost)
                yield (start, end)

    @staticmethod
    def _is_participle_phrase(token: spacy.tokens.token.Token) -> bool:
        return token.pos_ == "VERB" and any(node != token for node in token.subtree)

    @staticmethod
    def _is_skipped_pos(token: spacy.tokens.token.Token) -> bool:
        return (token.pos_ in SpanExtractor.skipped_pos
                or (token.pos_ == "PUNCT" and token.text not in SpanExtractor.allowed_punct))

class Doc:

    span_extractor = SpanExtractor()

    def __init__(self,
                 name: str,
                 data: dict,
                 semantics: dict = None,
                 *,
                 tokenizer: transformers.PreTrainedTokenizerBase,
                 extract_all_spans: bool = False,
                 ):
        self.name = name
        self.raw_data = data
        self.semantics = semantics
        self.encoding = tokenizer(data["text"],
                                  add_special_tokens=True,
                                  padding=True,
                                  truncation=True,
                                  max_length=max_text_length,
                                  return_tensors="pt",
                                  return_attention_mask=True,
                                  return_overflowing_tokens=True,
                                  return_offsets_mapping=True,
                                  return_special_tokens_mask=True)
        self.flattening_mask = (self.encoding.special_tokens_mask == 0)
        self.flat_offset_mapping = self.encoding.offset_mapping[self.flattening_mask].tolist()
        self._start_mapping: Dict[int, int] = {}
        self._end_mapping: Dict[int, int] = {}
        for token_i, (char_start, char_end) in enumerate(self.flat_offset_mapping):
            if char_start != char_end:
                if char_start in self._start_mapping or char_end in self._end_mapping:
                    logging.warning(f"{self.name}: "
                                    f"overlapping subtoken {token_i} at {(char_start, char_end)}; "
                                    f"existing subtokens: {self._start_mapping.get(char_start)}, "
                                    f"{self._end_mapping.get(char_end)}")
                    continue
                self._start_mapping[char_start] = token_i
                self._end_mapping[char_end] = token_i

        self.entities = self._match_entities_to_tokens()

        self._all_spans = None
        if extract_all_spans:
            self._all_spans = self._extract_all_spans()

        self.spans_classes = None
        self.sem_classes = None
        if self.semantics != None:
            self.spans_classes, self.sem_classes = self.match_sem_class_to_span()
            logging.warning("semantic classes: " + str(self.sem_classes))
            self.sem_classes_sizes = Counter(self.spans_classes.values())

    def match_sem_class_to_span(self):
        spans_classes = {}
        sem_classes = []
        for obj in self.semantics:
            start = obj["location"]["start"]
            end = obj["location"]["end"]
            char_span = (start, end)
            token_span = self.char_span_to_tokens((start, end))
            if token_span:
                sem_class = obj["value"]
                spans_classes[token_span] = sem_class
                if sem_class not in sem_classes:
                    sem_classes.append(sem_class)
                if token_span not in self._all_spans:
                    self._all_spans.append(token_span)
                logging.warning(f"{self.name}: adding classified entity {char_span}: {data['text'][start:end]}")
            else:
                logging.warning(f"{self.name}: skipping classified entity {char_span}: {data['text'][start:end]}")
        return spans_classes, sem_classes

    @property
    def all_spans(self) -> Iterator[Tuple[int, int]]:
        if self._all_spans is None:
            self._all_spans = self._extract_all_spans()
        return iter(self._all_spans)

    def char_span_to_tokens(self, span: Tuple[int, int]) -> Optional[Tuple[int, int]]:
        char_start, char_end = span
        token_start = self._start_mapping.get(char_start)
        token_end = self._end_mapping.get(char_end)
        if token_start is not None and token_end is not None:
            return (token_start, token_end + 1)
        logging.warning(f"{self.name}: skipping span {(char_start, char_end)}, "
                        f"{repr(self.raw_data['text'][char_start: char_end])}")

    def token_span_to_chars(self, span: Tuple[int, int]) -> Tuple[int, int]:
        token_start, token_end = span
        char_start = self.flat_offset_mapping[token_start][0]
        char_end = self.flat_offset_mapping[token_end - 1][1]
        return (char_start, char_end)

    def _extract_all_spans(self) -> List[Tuple[int, int]]:
        out = []
        for char_span in self.span_extractor(self.raw_data["text"]):
            token_span = self.char_span_to_tokens(char_span)
            if token_span:
                out.append(token_span)
        return out

    def _match_entities_to_tokens(self) -> List[List[Tuple[int, int]]]:
        char_entities = self.raw_data["entities"]
        token_entities = []

        for char_entity in char_entities:
            token_entity = []
            for char_start, char_end in char_entity:
                token_span = self.char_span_to_tokens((char_start, char_end))
                if token_span:
                    token_entity.append(token_span)
            if token_entity:
                token_entities.append(token_entity)
            else:
                logging.warning(f"{self.name}: skipping entity {char_entity}")

        return token_entities


ADD_PUNCTUATION_PRE_TOKENIZER = {global_encoder_model_name, }


def load_tokenizer(model_name: str) -> transformers.AutoTokenizer:
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
    if model_name in ADD_PUNCTUATION_PRE_TOKENIZER:
        tokenizer._tokenizer.pre_tokenizer = pre_tokenizers.Sequence([pre_tokenizers.Punctuation(),
                                                                      tokenizer._tokenizer.pre_tokenizer])
    return tokenizer


EPS = 1e-8


class LEAResult(NamedTuple):
    precision: float
    precision_weight: float
    recall: float
    recall_weight: float


class CorefModel(pl.LightningModule):
    def __init__(self,
                 encoder_model_name: str = global_encoder_model_name,
                 dropout_rate: float = 0.3,
                 k: int = 50,
                 max_batches_train: Optional[int] = None,
                 **discarded_kwargs):
        super().__init__()
        self.save_hyperparameters("encoder_model_name", "dropout_rate", "k", "max_batches_train")

        self.encoder = AutoModel.from_pretrained(encoder_model_name)

        self.token_importance_linear = torch.nn.Linear(self.encoder.config.hidden_size, 1)
        self.span_dropout = torch.nn.Dropout(dropout_rate)

        self.coarse_bilinear = torch.nn.Linear(self.encoder.config.hidden_size, self.encoder.config.hidden_size)
        self.coarse_dropout = torch.nn.Dropout(dropout_rate)

        self.fine_linear = torch.nn.Sequential(
            torch.nn.Linear(self.encoder.config.hidden_size * 3, self.encoder.config.hidden_size),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout_rate),
            torch.nn.Linear(self.encoder.config.hidden_size, 1)
        )

        self.k = k
        self.max_batches_train = max_batches_train

    @staticmethod
    def add_model_specific_args(parent_parser):
        parser = parent_parser.add_argument_group("CorefModel")
        parser.add_argument("--encoder_model_name", default=global_encoder_model_name)
        parser.add_argument("--dropout_rate", type=float, default=0.3)
        parser.add_argument("--k", type=int, default=50)
        parser.add_argument("--max_batches_train", type=int)
        return parent_parser

    def forward(self, doc: Doc) -> Tuple[torch.Tensor, torch.Tensor]:

        # Encoding spans ################

        # The input batches of the document are processed independently by a BERT-like model,
        # then the last hidden states of all the batch outputs are taken, exluding special tokens (cls, sep and pad)
        # This gives us embs - [n_tokens, emb] matrix with token embeddings
        input_ids, attention_mask = doc.encoding.input_ids.to(self.device), doc.encoding.attention_mask.to(self.device)
        if not self.training or self.max_batches_train is None or len(input_ids) <= self.max_batches_train:
            embs = self.encoder(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state
        else:
            embs = torch.zeros((*input_ids.shape, self.encoder.config.hidden_size), device=self.device)
            selected_batches = torch.rand(len(input_ids)).topk(self.max_batches_train).indices
            embs[selected_batches] = self.encoder(input_ids=input_ids[selected_batches],
                                                  attention_mask=attention_mask[selected_batches]).last_hidden_state
        embs = embs[doc.flattening_mask]      # Убираем специальные токены                                              # [n_tokens, emb]

        if doc.sem_classes != None and len(doc.sem_classes) > 0:
            all_scores = None
            all_indices = None
            for sem_class in doc.sem_classes:
                # We transform a matrix of span starts and ends into an [n_spans, n_tokens] boolean mask,
                # which for i-th row will have 1 at positions of tokens that are part of the i-th span
                spans = torch.tensor(list(doc.all_spans), dtype=torch.long, device=self.device)     # [n_spans, 2]
                indices = torch.arange(0, len(embs), device=self.device).unsqueeze(0).expand(len(spans), len(embs))
                span_mask = (indices >= spans[:, 0].unsqueeze(1)) * (indices < spans[:, 1].unsqueeze(1))
                span_mask = span_mask.to(torch.float)
                span_mask = torch.log(span_mask)
                # logging.warning("span mask: " + str(span_mask))

                # Each token representation is passed through trainable token importance linear layer
                # The obtained scores are then softmaxed for each span
                # This way, if a span consists of one token, this token's embeddings will have 1.0 of the weight
                # While, for example, for a two-token span with scores of [1.2, 2.6] the weights will be [0.2, 0.8]
                token_scores = self.token_importance_linear(embs).squeeze(1)                        # [n_tokens]
                token_scores = token_scores.unsqueeze(0).expand(len(spans), len(embs))              # [n_spans, n_tokens]
                token_scores = torch.softmax(token_scores + span_mask, dim=1)
                for span, span_class in doc.spans_classes.items():
                    if (span_class != sem_class):
                        span_index = doc._all_spans.index(span)
                        token_scores[span_index] *= 0
                        # spans = torch.cat((spans[:span_index],spans[span_index+1:]))
                logging.warning("token scores: " + str(token_scores))

                # Span representations are obtained as weighted sums of the token representations of the span
                embs = token_scores.mm(embs)                                                        # [n_spans, emb]
                logging.warning("span embeddings: " + str(embs))
                embs = self.span_dropout(embs)

                # Coarse span pair scoring ######

                # We set to -inf the scores of all span pairs (i, j) where i <= j (we only predict right to left links)
                pair_mask = torch.arange(0, len(embs), device=self.device)
                pair_mask = pair_mask.unsqueeze(1) - pair_mask.unsqueeze(0)
                pair_mask = torch.log((pair_mask > 0).to(torch.float))                              # [n_spans, n_spans]

                # Coarse coreference scores are obtained as S ⋅ W ⋅ S.T,
                #   where S is the matrix of span representations
                #   and W is a matrix of trainable weights
                # The pair mask is added to the scores to mask the links between undesired positions
                coarse_scores = self.coarse_dropout(self.coarse_bilinear(embs)).mm(embs.T)
                coarse_scores = pair_mask + coarse_scores                                           # [n_spans, n_spans]

                # Top scoring antecedents are taken further for expensive fine span pair scoring
                k=min(int(self.k / len(doc.sem_classes)), len(coarse_scores))
                top_scores, top_indices = torch.topk(coarse_scores,                                 # [n_spans, n_ants]
                                                    # k=min(int(self.k / len(doc.spans_classes) * doc.sem_classes_sizes[sem_class]), len(coarse_scores)),
                                                    k,
                                                    dim=1, sorted=False)

                # print("local k = " + str(self.k))
                # print("local k = " + str(k))
                # print("spans with classes = " + str(len(doc.spans_classes)))
                # print("class size = " + str(doc.sem_classes_sizes[sem_class]))
                # print("classes count = " + str(len(doc.sem_classes)))

                logging.warning(f"top scores for {sem_class}: " + str(top_scores))
                logging.warning(f"top indices for {sem_class}: " + str(top_indices))

                # Fine span pair scoring ########

                # Fine scores are obtained by first building the pair matrix: a concatenation of the following:
                #   a_spans: embeddings of all spans in the documents
                #   b_spans: embeddings of top-k antecedents for each span of the document
                #   similarity: element-wise product of a_spans and b_spans
                a_spans = embs.unsqueeze(1).expand(embs.shape[0], top_scores.shape[1], embs.shape[1])
                b_spans = embs[top_indices]
                similarity = a_spans * b_spans
                pair_matrix = torch.cat((a_spans, b_spans, similarity), dim=2)                  # [n_spans, n_ants, pair_emb]

                # The resulting pair matrix is passed through dense layers
                fine_scores = self.fine_linear(pair_matrix).squeeze(2)                          # [n_spans, n_ants]

                # Fine scores and coarse scores are added together (important for training)
                if all_scores == None:
                    all_scores = fine_scores + top_scores
                    all_indices = top_indices
                else:
                    torch.cat((all_scores, fine_scores + top_scores), dim=1)
                    torch.cat((all_indices, top_indices), dim=1)

            logging.warning("all scores: " + str(all_scores))
            logging.warning("all indices: " + str(all_indices))
            return all_scores, all_indices
        else:
            # We transform a matrix of span starts and ends into an [n_spans, n_tokens] boolean mask,
            # which for i-th row will have 1 at positions of tokens that are part of the i-th span
            spans = torch.tensor(list(doc.all_spans), dtype=torch.long, device=self.device)     # [n_spans, 2]
            indices = torch.arange(0, len(embs), device=self.device).unsqueeze(0).expand(len(spans), len(embs))
            span_mask = (indices >= spans[:, 0].unsqueeze(1)) * (indices < spans[:, 1].unsqueeze(1))
            span_mask = torch.log(span_mask.to(torch.float))
            # logging.warning("span mask: " + str(span_mask))

            # Each token representation is passed through trainable token importance linear layer
            # The obtained scores are then softmaxed for each span
            # This way, if a span consists of one token, this token's embeddings will have 1.0 of the weight
            # While, for example, for a two-token span with scores of [1.2, 2.6] the weights will be [0.2, 0.8]
            token_scores = self.token_importance_linear(embs).squeeze(1)                        # [n_tokens]
            token_scores = token_scores.unsqueeze(0).expand(len(spans), len(embs))              # [n_spans, n_tokens]
            token_scores = torch.softmax(token_scores + span_mask, dim=1)

            # Span representations are obtained as weighted sums of the token representations of the span
            embs = token_scores.mm(embs)                                                        # [n_spans, emb]
            embs = self.span_dropout(embs)

            # Coarse span pair scoring ######

            # We set to -inf the scores of all span pairs (i, j) where i <= j (we only predict right to left links)
            pair_mask = torch.arange(0, len(embs), device=self.device)
            pair_mask = pair_mask.unsqueeze(1) - pair_mask.unsqueeze(0)
            pair_mask = torch.log((pair_mask > 0).to(torch.float))                              # [n_spans, n_spans]

            # Coarse coreference scores are obtained as S ⋅ W ⋅ S.T,
            #   where S is the matrix of span representations
            #   and W is a matrix of trainable weights
            # The pair mask is added to the scores to mask the links between undesired positions
            coarse_scores = self.coarse_dropout(self.coarse_bilinear(embs)).mm(embs.T)
            coarse_scores = pair_mask + coarse_scores                                           # [n_spans, n_spans]

            # Top scoring antecedents are taken further for expensive fine span pair scoring
            top_scores, top_indices = torch.topk(coarse_scores,                                 # [n_spans, n_ants]
                                                k=min(self.k, len(coarse_scores)),
                                                dim=1, sorted=False)

            # Fine span pair scoring ########

            # Fine scores are obtained by first building the pair matrix: a concatenation of the following:
            #   a_spans: embeddings of all spans in the documents
            #   b_spans: embeddings of top-k antecedents for each span of the document
            #   similarity: element-wise product of a_spans and b_spans
            a_spans = embs.unsqueeze(1).expand(embs.shape[0], top_scores.shape[1], embs.shape[1])
            b_spans = embs[top_indices]
            similarity = a_spans * b_spans
            pair_matrix = torch.cat((a_spans, b_spans, similarity), dim=2)                  # [n_spans, n_ants, pair_emb]

            # The resulting pair matrix is passed through dense layers
            fine_scores = self.fine_linear(pair_matrix).squeeze(2)                          # [n_spans, n_ants]

            # logging.warning(f"top scores: " + str(top_scores))
            # logging.warning(f"top indices: " + str(top_indices))

            # Fine scores and coarse scores are added together (important for training)
            return fine_scores + top_scores, top_indices

    @staticmethod
    def lea(a_clusters: List[List[Tuple[int, int]]],
            b_clusters: List[List[Tuple[int, int]]]) -> LEAResult:
        recall, r_weight = CorefModel._lea(a_clusters, b_clusters)
        precision, p_weight = CorefModel._lea(b_clusters, a_clusters)
        return LEAResult(precision, p_weight, recall, r_weight)

    @staticmethod
    def _lea(key: List[List[Tuple[int, int]]],
             response: List[List[Tuple[int, int]]]) -> Tuple[float, float]:
        """ See aclweb.org/anthology/P16-1060.pdf. """
        response_clusters = [set(cluster) for cluster in response]
        response_map = {mention: cluster
                        for cluster in response_clusters
                        for mention in cluster}
        importances = []
        resolutions = []
        for entity in key:
            size = len(entity)
            if size == 1:  # entities of size 1 are not annotated
                continue
            importances.append(size)
            correct_links = 0
            for i in range(size):
                for j in range(i + 1, size):
                    correct_links += int(entity[i] in response_map.get(entity[j], {}))
            resolutions.append(correct_links / (size * (size - 1) / 2))
        res = sum(imp * res for imp, res in zip(importances, resolutions))
        weight = sum(importances)
        return res, weight

    def loss(self, doc: Doc, top_scores: torch.Tensor, top_indices: torch.Tensor) -> torch.Tensor:
        span2entity = {span: i for i, entity in enumerate(doc.entities, start=1) for span in entity}
        entity_ids = torch.tensor([span2entity.get(span, 0) for span in doc.all_spans], device=self.device)

        valid_pair_map = (top_scores > float('-inf'))
        y = entity_ids[top_indices] * valid_pair_map
        y[y == 0] = -1
        y = (y == entity_ids.unsqueeze(1))
        y = torch.cat((y, torch.full((len(y), 1), False, device=y.device)), dim=1)
        y[y.sum(dim=1) == 0, -1] = True
        y = y.to(torch.float)  # [n_spans, k + 1]

        top_scores = torch.cat((top_scores, torch.zeros((len(top_scores), 1), device=top_scores.device)), dim=1)

        gold = torch.logsumexp(top_scores + torch.log(y.to(torch.float)), dim=1)
        pred = torch.logsumexp(top_scores, dim=1)
        return (pred - gold).mean()

    def predict(self, doc: Doc) -> List[List[Tuple[int, int]]]:
        top_scores, top_indices = self(doc)
        positive_scores = (top_scores > 0).detach().cpu().numpy()
        best_ants = top_scores.argmax(dim=1).cpu().numpy()
        indices_map = top_indices.cpu().numpy()

        spans = list(doc.all_spans)
        links = []
        for a_idx, b_ant_idx in enumerate(best_ants):
            if positive_scores[a_idx, b_ant_idx]:
                b_idx = indices_map[a_idx, b_ant_idx]
                links.append((spans[a_idx], spans[b_idx]))

        span2entity = {}

        def get_entity(span: Tuple[int, int]) -> List[Tuple[int, int]]:
            if span not in span2entity:
                span2entity[span] = [span]
            return span2entity[span]

        for source, target in links:
            source_entity, target_entity = get_entity(source), get_entity(target)
            if source_entity is not target_entity:
                source_entity.extend(target_entity)
                for span in target_entity:
                    span2entity[span] = source_entity

        ids = set()
        entities = []
        for entity in span2entity.values():
            if id(entity) not in ids:
                ids.add(id(entity))
                entities.append(entity)

        return sorted(sorted(entity) for entity in entities)

    def run(self, doc: Doc) -> torch.Tensor:
        top_scores, top_indices = self(doc)
        return self.loss(doc, top_scores, top_indices)

    def training_step(self, batch: List[Doc], batch_idx: int):
        if len(batch) > 1:
            loss = torch.cat([self.run(doc) for doc in batch]).sum()
        else:
            loss = self.run(batch[0])
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch: List[Doc], batch_idx: int):
        scores = []
        for doc in batch:
            scores.append(self.lea(self.predict(doc), doc.entities))
        return scores

    def validation_epoch_end(self, outputs: List[List[LEAResult]]):  # type: ignore[override]
        precision = 0.0
        precision_weight = 0.0
        recall = 0.0
        recall_weight = 0.0
        for output in outputs:
            for result in output:
                precision += result.precision
                precision_weight += result.precision_weight
                recall += result.recall
                recall_weight += result.recall_weight

        total_precision = precision / (precision_weight + EPS)
        total_recall = recall / (recall_weight + EPS)
        f1 = (total_precision * total_recall) / (total_precision + total_recall + EPS) * 2
        self.log("val_lea", f1)

    def configure_optimizers(self):
        coref_parameters = []
        for submodule in self.children():
            if submodule is not self.encoder:
                coref_parameters.extend(submodule.parameters())
        return torch.optim.Adam([
            {"params": self.encoder.parameters(), "lr": 1e-5},
            {"params": coref_parameters, "lr": 3e-4}
        ])


class CorefDocs(torch.utils.data.Dataset):
    def __init__(self, path: str, tokenizer: transformers.PreTrainedTokenizerFast):
        self.docs = []
        for entry in os.scandir(path):
            if entry.name.endswith(".json"):
                with open(entry.path, mode="r", encoding="utf8") as f:
                    data = json.load(f, strict=False)
                    logging.warning(entry.name)
                    # logging.warning(data["text"])
                    if len(data["text"]) > 0:
                        if data["text"][0] == " ":
                            data["text"] = data["text"].replace(" ", "", 1).replace('\n', '')
                            logging.warning("Replace first space")

                    data = json.loads(json.dumps(data).replace('\\t', '').replace('\t', '').replace('\\r', '').replace('\r', '').replace('\\0', '').replace('\0', ''), strict=False)
                    # data = json.loads(json.dumps(data), strict=False)

                    self.docs.append(Doc(entry.name, data, tokenizer=tokenizer, extract_all_spans=True))


    def __getitem__(self, idx: int) -> Doc:
        return self.docs[idx]

    def __len__(self) -> int:
        return len(self.docs)


# Download and split datasets

In [ ]:
import argparse
import json
import os
import random
import shutil
import sys
import logging
from typing import List, Tuple

logging.disable(level=logging.NOTSET)

parser = argparse.ArgumentParser()
parser.add_argument("--data_dir", default="./rucoco/data/train")
parser.add_argument("--out_dir", default="./rucoco/data/split_data")
parser.add_argument("--val_fraction", type=float, default=0.10)
parser.add_argument("--test_fraction", type=float, default=0.10)
parser.add_argument("-f", action="store_true", help="Overwrite out_dir if it already exists.")
args = parser.parse_args([])

assert 0.01 <= args.val_fraction <= 0.2
assert 0.1 <= args.test_fraction <= 0.3

if os.path.exists(args.out_dir):
    if not args.f:
        print(f"The out directory {args.out_dir} already exists. Use -f to overwrite it.")
        sys.exit(1)
    shutil.rmtree(args.out_dir)
for split in "train", "val", "test":
    os.makedirs(os.path.join(args.out_dir, split))

datafiles: List[Tuple[os.DirEntry, int]] = []  # [(path, n_words), ...]
for entry in os.scandir(args.data_dir):
    if entry.name.endswith(".json"):
        logging.warn(entry.name)
        with open(entry, encoding="utf8") as f:
            length = json.load(f, strict=False)["text"].split().__len__()
        datafiles.append((entry, length))

total_length = sum(length for _, length in datafiles)
test_length = int(total_length * args.test_fraction)
val_length = int(total_length * args.val_fraction)

random.shuffle(datafiles)
current_test_length = 0
while current_test_length < test_length:
    entry, length = datafiles.pop()
    dst_path = os.path.join(args.out_dir, "test", entry.name)
    shutil.copyfile(entry.path, dst_path)
    current_test_length += length

current_val_length = 0
while current_val_length < val_length:
    entry, length = datafiles.pop()
    dst_path = os.path.join(args.out_dir, "val", entry.name)
    shutil.copyfile(entry.path, dst_path)
    current_val_length += length

for entry, length in datafiles:
    dst_path = os.path.join(args.out_dir, "train", entry.name)
    shutil.copyfile(entry.path, dst_path)

print(f"Train: {total_length - current_val_length}\n"
      f"Val:   {current_val_length}\n"
      f"Test:   {current_test_length}")


/tmp/ipykernel_170629/4078494547.py:34: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  logging.warn(entry.name)
IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
# from google.colab import files
# files.download('/content/split_data/train/book_2111.json')

# Tokenize datasets

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
tokenizer = load_tokenizer(global_encoder_model_name)

In [ ]:
logging.disable()

In [ ]:
logging.disable(level=logging.NOTSET)

In [ ]:
import logging
logger = logging.getLogger('my_logger')

# logging.basicConfig(
#     filename='./rucoco/app.log', # write to this file
#     filemode='w', # open in append mode
#     # format='%(name)s - %(levelname)s - %(message)s',
#     force=True,
#     level=logging.DEBUG
#     )

# logging.basicConfig()

logging.debug('This will get logged to a file')

In [ ]:
# train_data_test = CorefDocs("./rucoco/data/data_test", tokenizer)

In [ ]:
train_data = CorefDocs(os.path.join("./rucoco/data/split_data", "train"), tokenizer)

In [ ]:
val_data = CorefDocs(os.path.join("./rucoco/data/split_data", "val"), tokenizer)

In [ ]:
from pickle5 import dump

In [ ]:
f = open("./rucoco/data/tokenized_data/train_data_lf_large.txt","wb")
dump(train_data, f)
f.close()

In [ ]:
f = open("./rucoco/data/tokenized_data/val_data_lf_large.txt","wb")
dump(val_data, f)
f.close()

In [ ]:
for span in train_data[0].all_spans:
  print(train_data[0].token_span_to_chars(span))

# Load tokenized datasets

In [ ]:
from pickle5 import load

In [ ]:
f = open("./rucoco/data/tokenized_data/train_data_lf_large.txt","rb")
train_data = load(f)
f.close()

In [ ]:
f = open("./rucoco/data/tokenized_data/val_data_lf_large.txt","rb")
val_data = load(f)
f.close()

#Train

In [ ]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
import argparse
from copy import deepcopy
import json

import pytorch_lightning as pl
import torch
import transformers
from torch.utils.data import Subset
from torch.utils.data import DataLoader
import shutil


def zip_directory(source_dir, zip_filename):
    shutil.make_archive(zip_filename, 'zip', source_dir)


class BasicCheckpointIO(pl.plugins.io.TorchCheckpointIO):
    def save_checkpoint(self, checkpoint, path, storage_options=None):
        os.makedirs(os.path.dirname(path), exist_ok=True)
        torch.save(checkpoint, path)


In [ ]:
model = CorefModel(k=50, max_batches_train=4)

Some weights of the model checkpoint at kazzand/ru-longformer-large-4096 were not used when initializing LongformerModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerModel were not initialized from the model checkpoint at kazzand/ru-longformer-large-4096 and are newly initialized: ['longformer.pooler.dense.bias', 'longformer.pooler.dense.weight']
You should probably TRAIN this mode

In [ ]:

collate_fn = lambda x: deepcopy(x)
#subset_indices = range(1000)  # Choose the indices of the samples you want
# train_subset = Subset(train_data, subset_indices)
# val_subset = Subset(val_data, subset_indices)

train_data_loader = torch.utils.data.DataLoader(train_data,
                                                shuffle=True,
                                                collate_fn=collate_fn,
                                                num_workers=4,
                                                batch_size=1,
                                                # pin_memory=True
                                                )

val_data_loader = torch.utils.data.DataLoader(val_data,
                                              collate_fn=collate_fn
                                              )

early_stopping = pl.callbacks.early_stopping.EarlyStopping(monitor="val_lea",
                                                          min_delta=0.01,
                                                          mode="max",
                                                          patience=10,
                                                        #   verbose=True
                                                          )

checkpointing = pl.callbacks.ModelCheckpoint(monitor="val_lea",
                                             mode="max",
                                             filename="{epoch:02d}-{val_lea:.3f}",
                                             every_n_epochs=1,
                                             save_last=True
                                            #  save_weights_only=True
                                             )
trainer = pl.Trainer(
                    #  plugins=[BasicCheckpointIO()],
                     default_root_dir='./rucoco',
                     accelerator="gpu",
                     devices=1,
                     callbacks=[early_stopping, checkpointing],
                    #  log_every_n_steps=2,
                    #  min_epochs=5
                     )
# trainer = pl.Trainer.from_argparse_args(args, callbacks=[early_stopping, checkpointing])

GPU available: True (cuda), used: True
TPU available: None, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
# checkpoint_path = ".rucoco/lightning_logs/version_5/checkpoints/last.ckpt"
# model = model.load_from_checkpoint(checkpoint_path)

In [ ]:
logging.disable(level=logging.NOTSET)

In [ ]:
torch.cuda.empty_cache()
trainer.fit(model, train_data_loader, val_data_loader)

You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]

  | Name                    | Type            | Params
------------------------------------------------------------
0 | encoder                 | LongformerModel | 434 M 
1 | token_importance_linear | Linear          | 1.0 K 
2 | span_dropout            | Dropout         | 0     
3 | coarse_bilinear         | Linear          | 1.0 M 
4 | coarse_dropout          | Dropout         | 0     
5 | fine_linear             | Sequential      | 3.1 M 
------------------------------------------------------------
438 M     Trainable params
0         Non-trainable params
438 M     Total params
1,755

Sanity Checking: 0it [00:00, ?it/s]

/home/shuvalov/.conda/envs/env/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
# torch.save(model, "./model.pt")

#Make raw test dataset

In [ ]:
import json
import os

# Define the directory where your JSON files are located
json_dir = './rucoco/data/split_data/test'
txt_dir = './rucoco/data/split_data/test_raw'
os.mkdir(txt_dir)
# Iterate over all files in the directory
for filename in os.listdir(json_dir):
    if filename.endswith('.json'):
        file_path = os.path.join(json_dir, filename)

        # Load the JSON data from the current file
        with open(file_path,  mode="r", encoding="utf8") as json_file:
            data = json.load(json_file, strict=False)
            if len(data["text"]) > 0:
                if data["text"][0] == " ":
                    data["text"] = data["text"].replace(" ", "", 1).replace('\n', '')
                    logging.debug("Replace first space")
                logging.debug(data["text"])
                data["text"] = data["text"].replace('\\t', '').replace('\t', '').replace('\\r', '').replace('\r', '').replace('\\0', '').replace('\0', '')
                data = json.loads(json.dumps(data), strict = False)

        # Extract the 'text' field value
        text_value = data.get('text', '')

        # Create a corresponding txt file with the same name as the JSON file
        txt_filename = filename.replace('.json', '.txt')
        txt_file_path = os.path.join(txt_dir, txt_filename)

        # Save the 'text' value to the txt file
        with open(txt_file_path, 'w+') as txt_file:
            txt_file.write(text_value)

#Predict

In [ ]:
from tokenizers import pre_tokenizers
import transformers

ADD_PUNCTUATION_PRE_TOKENIZER = {global_encoder_model_name, }


def load_tokenizer(model_name: str) -> transformers.AutoTokenizer:
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
    if model_name in ADD_PUNCTUATION_PRE_TOKENIZER:
        tokenizer._tokenizer.pre_tokenizer = pre_tokenizers.Sequence([pre_tokenizers.Punctuation(),
                                                                      tokenizer._tokenizer.pre_tokenizer])
    return tokenizer

In [ ]:
import argparse
import json
import os

import torch
from tqdm import tqdm

# torch.cuda.empty_cache()

data_dir = "./rucoco/data/split_data/test_raw"
out_dir = "./rucoco/result/lf_large"
# weights = "./checkpoints/lightning_logs/version_1/checkpoints/epoch=04-val_lea=0.683.ckpt"
device = "cuda:0"
# device = "cpu"

In [ ]:
model = CorefModel()
model = model.load_from_checkpoint("./rucoco/lightning_logs/version_13/checkpoints/last.ckpt")

# model = CorefModel()
# model = model.load_from_checkpoint(weights)

Some weights of the model checkpoint at kazzand/ru-longformer-tiny-16384 were not used when initializing LongformerModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerModel were not initialized from the model checkpoint at kazzand/ru-longformer-tiny-16384 and are newly initialized: ['longformer.pooler.dense.weight', 'longformer.pooler.dense.bias']
You should probably TRAIN this mode

In [ ]:
model.to(device)
model.eval()

In [ ]:
tokenizer = load_tokenizer(model.hparams["encoder_model_name"])

os.makedirs(out_dir, exist_ok=True)

entries = [entry for entry in os.scandir(data_dir) if entry.name.endswith(".txt")]
with torch.no_grad():
    i = 0
    for entry in tqdm(entries, leave=True):
        i += 1
        # if i == 200:
        #     break;
        with open(entry.path, mode="r", encoding="utf8") as f:
            data = {"text": f.read(), "entities": [], "includes": []}
            doc = Doc(entry.name, data, tokenizer=tokenizer, extract_all_spans=True)
            char_entities = [
                [doc.token_span_to_chars(span) for span in token_entity]
                for token_entity in model.predict(doc)
            ]
            data["entities"] = char_entities
            data["includes"] = [[] for _ in char_entities]
            out_name = os.path.splitext(entry.name)[0] + ".json"
            with open(os.path.join(out_dir, out_name), mode="w", encoding="utf8") as f:
                json.dump(data, f, ensure_ascii=False)
    print("Files: ", i)

#Evaluate

In [ ]:
def read_json(file):
    data = json.load(file, strict=False)
    if len(data["text"]) > 0:
        if data["text"][0] == " ":
            data["text"] = data["text"].replace(" ", "", 1).replace('\n', '')

    return json.loads(json.dumps(data).replace('\\t', '').replace('\t', '').replace('\\r', '').replace('\r', '').replace('\\0', '').replace('\0', ''), strict=False)

In [ ]:
from collections import defaultdict
import json
import os
import sys
from typing import *
import logging

Span = Tuple[int, int]

EPS = 1e-7


class DocumentPair(NamedTuple):
    filename: str
    dir_a: str
    dir_b: str


class ScoringException(Exception):
    pass


def agreement(pairs: Iterable[DocumentPair]) -> Tuple[float, float, float]:
    total_recall, total_r_weight = .0, .0
    total_precision, total_p_weight = .0, .0
    k = 0
    for pair in sorted(pairs):
        k += 1
        a = read_markup_dict(os.path.join(pair.dir_a, pair.filename))
        b = read_markup_dict(os.path.join(pair.dir_b, pair.filename))
        if a["text"] != b["text"]:
            raise ScoringException(f"mismatching texts for documents: {pair.filename} in {pair.dir_a} and {pair.dir_b}")

        a_clusters = [(spans, get_children(a, i))
                      for i, spans in enumerate(a["entities"])]
        b_clusters = [(spans, get_children(b, i))
                      for i, spans in enumerate(b["entities"])]

        recall, r_weight = _lea_children(a_clusters, b_clusters)
        precision, p_weight = _lea_children(b_clusters, a_clusters)

        total_recall += recall
        total_r_weight += r_weight
        total_precision += precision
        total_p_weight += p_weight
        print(pair.filename)

    recall = total_recall / (total_r_weight + EPS)
    precision = total_precision / (total_p_weight + EPS)
    print("Files: ", k)
    return f1(recall, precision), precision, recall


def f1(precision: float, recall: float, eps: float = 1e-7) -> float:
    return (precision * recall) / (precision + recall + eps) * 2


def get_children(data: dict, idx: int) -> List[Span]:
    """ Returns a list of all the immediate AND most distant children """
    children = set()
    for child_idx in data["includes"][idx]:
        children.update(data["entities"][child_idx])

    visited = set()
    stack = list(data["includes"][idx])
    while stack:
        child_idx = stack.pop()
        visited.add(child_idx)
        if not data["includes"][child_idx]:
            children.update(data["entities"][child_idx])
        else:
            for grandchild_idx in data["includes"][child_idx]:
                if grandchild_idx not in visited:
                    stack.append(grandchild_idx)

    return sorted(children)


def get_pairs_from_dir(path: str) -> List[DocumentPair]:
    entries = filter(lambda entry: entry.name.endswith(".json"),
                     recursive_scandir(path))
    name2paths = defaultdict(list)
    for entry in entries:
        name2paths[entry.name].append(entry.path)

    pairs = []
    for name, paths in name2paths.items():
        if len(paths) == 1:
            raise ScoringException(f"No matching document for {paths[0]}")
        elif len(paths) > 2:
            raise ScoringException(f"Too many matching documents: {', '.join(paths)}")
        else:
            pairs.append(
                DocumentPair(name, *(os.path.dirname(path) for path in paths))
            )
    return pairs


def get_pairs_from_two_dirs(a: str,
                            b: str) -> List[DocumentPair]:
    a_files = set(get_relative_paths(a))
    b_files = set(get_relative_paths(b))
    common_files = a_files & b_files

    for file in a_files - common_files:
        raise ScoringException(f"No matching document for {os.path.join(a, file)}")
    for file in b_files - common_files:
        raise ScoringException(f"No matching document for {os.path.join(b, file)}")
    return [DocumentPair(filename, a, b) for filename in common_files]


def get_relative_paths(path: str) -> Iterator[str]:
    return map(lambda entry: os.path.relpath(entry.path, path),
               filter(lambda entry: entry.name.endswith(".json"),
                      recursive_scandir(path)))


def read_markup_dict(path: str) -> dict:
    with open(path, mode="r", encoding="utf8") as f:
        markup_dict = read_json(f)
    markup_dict["entities"] = [[tuple(span) for span in entity]
                               for entity in markup_dict["entities"]]
    return markup_dict


def recursive_scandir(path: str) -> Iterator[os.DirEntry]:
    for entry in os.scandir(path):
        if entry.is_dir():
            yield from recursive_scandir(entry.path)
        else:
            yield entry


def _lea_children(key: List[Tuple[List[Span], List[Span]]],
                  response: List[Tuple[List[Span], List[Span]]]
                  ) -> Tuple[float, float]:
        response_clusters = [set(cluster) for cluster, _ in response]
        response_map = {mention: cluster
                        for cluster in response_clusters
                        for mention in cluster}
        response_children_map = defaultdict(set)
        for cluster, children in response:
            for mention in children:
                response_children_map[mention].update(cluster)

        importances = []
        resolutions = []
        for entity, children in key:
            size = len(entity)
            if size > 1:  # entities of size 1 are not annotated
                importances.append(size)
                correct_links = 0
                for i in range(size):
                    for j in range(i + 1, size):
                        correct_links += int(entity[i]
                                            in response_map.get(entity[j], {}))
                resolutions.append(correct_links / (size * (size - 1) / 2))

            if not children:
                continue
            importances.append(len(children))
            correct_links = 0
            for mention in entity:
                for child in children:
                    correct_links += int(mention in response_children_map.get(child, {}))
            resolutions.append(correct_links / (size * len(children)))

        res = sum(imp * res for imp, res in zip(importances, resolutions))
        weight = sum(importances)
        return res, weight


In [ ]:
ref_dir = json_dir = './rucoco/data/split_data/test'
res_dir = out_dir = './rucoco/result/lf_large'
output_dir = "./rucoco/evaluation"

scores_path = os.path.join(output_dir, "RuCoCo_lf_large_scores.txt")
print("Get pairs from two dirs")
pairs = get_pairs_from_two_dirs(ref_dir, res_dir)
print("Agreement")
f1_score, precision, recall = agreement(pairs)

os.makedirs(output_dir, exist_ok=True)
with open(scores_path, mode="w", encoding="utf8") as f:
    print(f"F1: {f1_score:.3f}", file=f)
    print(f"Precision: {precision:.3f}", file=f)
    print(f"Recall: {recall:.3f}", file=f)

# Results view

In [ ]:
!pip install colorama

In [ ]:
from colorama import Fore, Back, Style

In [ ]:
log_dir = "./rucoco/res_view/"
logs_json_dir = "./rucoco/res_view/view_json"
logs_txt_dir = "./rucoco/res_view/view_txt"

def print_log(log):
    print(log)

def print_log_in_line(log):
    print(log, end="")

single_indent = "\t"
indent = "\t\t"

def print_entity_spans(entity, text):
    print_log("")
    print_log_in_line("\t")
    print_log("Реальные упоминания сущности")
    print_log_in_line(indent)
    print_log(entity)
    i = 1
    for span in entity:
        print_log_in_line(indent + str(i) + ". ")
        i += 1
        print_adjacent_text(span[0], span[1], text)

def print_adjacent_text(span_start, span_end, text):
    start = span_start
    end = span_end
    delta = 80
    if (start - delta >= 0):
        start = start - delta
    else:
        start = 0

    if (end + delta < len(text)):
        end = end + delta
    else:
        end = len(text) - 1

    if '\n' in text[start : span_start]:
        start = text[0 : span_start].rfind('\n') + 1

    if '\n' in text[span_end : end]:
        end = span_end + text[span_end : end].index('\n')

    print_log_in_line(text[start : span_start])
    # print_log_in_line("[")
    print_log_in_line(Fore.LIGHTGREEN_EX + text[span_start : span_end])
    print_log_in_line(Style.RESET_ALL)
    # print_log_in_line("]")
    print_log(text[span_end : end])

def print_matched(matched_corefs, text):
    print_log("")
    print_log_in_line(single_indent)
    print_log("Правильнно разрешенные кореференции")
    print_log_in_line(single_indent)
    print_log("===========")
    for coref in matched_corefs:
        print_log_in_line(indent)
        print_log(coref)
        print_log_in_line(indent)
        print_adjacent_text(coref[0][0], coref[0][1], text)
        print_log_in_line(indent)
        print_adjacent_text(coref[1][0], coref[1][1], text)
        # print_log(text[coref[0][0] : coref[0][1]])
        # print_log(text[coref[1][0] : coref[1][1]])
        print_log("")
    print_log_in_line(single_indent)
    print_log("===========")
    print_log("")


def print_not_identified(not_identified_corefs, text, response_map):
    print_log_in_line(single_indent)
    print_log("Неразрешенные кореференции")
    print_log_in_line(single_indent)
    print_log("===========")
    for coref in not_identified_corefs:
        print_log_in_line(indent)
        print_log(coref)
        # print_log(text[coref[0][0] : coref[0][1]])
        # print_log(text[coref[1][0] : coref[1][1]])
        start = coref[0][0]
        end = coref[0][1]
        print_log_in_line(indent)
        print_adjacent_text(start, end, text)
        start = coref[1][0]
        end = coref[1][1]
        print_log_in_line(indent)
        print_adjacent_text(start, end, text)
        print_log("")
        print_log_in_line(indent)
        print_log("Предсказанный кластер второго упоминания")
        print_log_in_line(indent + "\t")
        cluster = response_map.get(coref[1], {})
        print_log(cluster)
        for span in cluster:
            print_log_in_line(indent + "\t")
            print_adjacent_text(span[0], span[1], text)
        print_log("")
        print_log("")

    print_log_in_line(single_indent)
    print_log("===========")
    print_log("")

def split_into_disjoint_subsets(key: List[Tuple[List[Span], List[Span]]],
        response: List[Tuple[List[Span], List[Span]]],
        text
        ) -> List[Tuple[List[Tuple[Span, Span]], List[Tuple[Span, Span]], List[Tuple[Span, Span]]]]:
        response_clusters = [set(cluster) for cluster, _ in response]
        response_map = {mention: cluster
                        for cluster in response_clusters
                        for mention in cluster}
        response_children_map = defaultdict(set)
        for cluster, children in response:
            for mention in children:
                response_children_map[mention].update(cluster)

        subsets_list: List[Tuple[List[Span], List[Span], List[Span]]] = []

        # for entity_num in range(len(key)):
        #     entity = key[entity_num][0]

        entity_count = 1
        for entity, children in key:
            size = len(entity)
            if size > 1:  # entities of size 1 are not annotated
                matched_coref : List[Tuple[Span, Span]] = []
                not_identified_coref : List[Tuple[Span, Span]] = []
                wrong_coref : List[Tuple[Span, Span]] = []

                for i in range(size):
                    for j in range(i + 1, size):
                        if (entity[i] in response_map.get(entity[j], {})):
                            matched_coref.append((entity[i], entity[j]))
                        else:
                            not_identified_coref.append((entity[i], entity[j]))

                # ищем кластер, которому соответствует первое и
                # правильно предсказанное упоминание сущности
                # if (len(matched_coref) > 0):
                    # predicted_spans = response_map[matched_coref[0]]
                    # wrong_spans = list(set(predicted_spans) - set(matched_coref))

                subsets_list.append((matched_coref, not_identified_coref, wrong_coref))

                print_log("Сущность " + str(entity_count))
                print_log("==================================")
                print_entity_spans(entity, text)
                print_matched(matched_coref, text)
                print_not_identified(not_identified_coref, text, response_map)
                print_log("==================================")
                print_log("")
                entity_count += 1
            # entity_num += 1
        return subsets_list

def log(pairs: Iterable[DocumentPair]):
    os.makedirs(log_dir, exist_ok=True)
    os.makedirs(logs_json_dir, exist_ok=True)
    os.makedirs(logs_txt_dir, exist_ok=True)
    i = 0
    for pair in sorted(pairs):
        a = read_markup_dict(os.path.join(pair.dir_a, pair.filename))
        b = read_markup_dict(os.path.join(pair.dir_b, pair.filename))
        if a["text"] != b["text"]:
            raise ScoringException(f"mismatching texts for documents: {pair.filename} in {pair.dir_a} and {pair.dir_b}")

        a_clusters = [(spans, get_children(a, i))
                      for i, spans in enumerate(a["entities"])]
        b_clusters = [(spans, get_children(b, i))
                      for i, spans in enumerate(b["entities"])]

        print_log("=================" + pair.filename + "=================")
        print_log(a["text"])
        with open(os.path.join(logs_json_dir, pair.filename), mode="w+", encoding="utf8") as f:
           json.dump(split_into_disjoint_subsets(a_clusters, b_clusters, a["text"]), f, ensure_ascii='False')
        print_log("=================" + pair.filename + " END=================")
        i += 1
        if (i == 10):
            break;

In [ ]:
pairs = get_pairs_from_two_dirs("./rucoco/data/split_data/test", "./rucoco/result")
log(pairs)

In [ ]:
def print_logs(ref_path, log_path):
    text = ""
    with open(ref_path, mode="r", encoding="utf8") as f:
        data = json.load(f, strict=False)
        text = data["text"]
        print_log(text)

    with open(log_path, mode="r", encoding="utf8") as f:
        logs = json.load(f, strict=False)
        # logging.warning(logs)
        for matched_corefs, not_identified_coref, wrong_corefs in logs:
            print_log("Matched")
            print_log("===========")
            for coref in matched_corefs:
                print_log(coref)
                print_log(text[coref[0][0] : coref[0][1]])
                print_log(text[coref[1][0] : coref[1][1]])
                print_log("")
            print_log("===========")
            print_log("")
            print_log("Not identified")
            print_log("===========")
            print_log("")
            for coref in not_identified_coref:
                print_log(coref)
                # print_log(text[coref[0][0] : coref[0][1]])
                # print_log(text[coref[1][0] : coref[1][1]])
                start = coref[0][0]
                end = coref[0][1]
                print_adjacent_text(start, end, text)
                start = coref[1][0]
                end = coref[1][1]
                print_adjacent_text(start, end, text)
                print_log("")
            print_log("===========")
            print_log("")


In [ ]:
print_logs("./rucoco/data/split_data/test/2000_finance_bujet_004.json", "./rucoco/res_view/view_json/2000_finance_bujet_004.json")

## Print text with highlighted entity mentions

In [ ]:
def print_entity(entity, text):
    print_log_in_line(indent)
    print_log(entity)
    i = 1
    for span in entity:
        print_log_in_line(indent + str(i) + ". ")
        i += 1
        print_adjacent_text(span[0], span[1], text)

def print_corefs(text_path):
    text = ""
    with open(text_path, mode="r", encoding="utf8") as f:
        data = json.load(f, strict=False)
        text = data["text"]
        print_log(text)

    i = 1
    for entity in data["entities"]:
        print_log("")
        print_log("Сущность " + str(i))
        print_entity(entity, text)
        i += 1
        print_log("")


def highlite_entity_mentions(entities, text : str) -> str:
    shift = 0
    for entity in entities:
        for span in entity:
            # text = text[:span[0] + shift] + Fore.LIGHTGREEN_EX + text[span[0] + shift:span[1] + shift] + Style.RESET_ALL + text[span[1] + shift:]
            # shift += len(Fore.GREEN) + len(Style.RESET_ALL)
            text = text[:span[0]] + text[span[0]:span[1]].upper() + text[span[1]:]
    return text

def print_highlited(text_path : str):
    text = ""
    with open(text_path, mode="r", encoding="utf8") as f:
        data = json.load(f, strict=False)
        text = highlite_entity_mentions(data["entities"], data["text"])
        print(text)

In [ ]:
print_corefs("./rucoco/kristina/kristinas_result/text_716918.json")



Как работает ChatGPT: объясняем на простом русском эволюцию языковых моделей с T9 до чуда / Habr


               Как работает ChatGPT: объясняем на простом русском эволюцию языковых моделей с T9 до чуда Level of difficulty  
    Easy
   Reading time  
    30 min
   Views  235K Open Data Science corporate blog Machine learning *Artificial Intelligence The future is here Natural Language Processing * 
    Review
        В последнее время нам почти каждый день рассказывают в новостях, какие очередные вершины покорили языковые нейросетки, и почему они уже через месяц совершенно точно оставят лично вас без работы. При этом мало кто понимает — а как вообще нейросети вроде ChatGPT работают внутри? Так вот, устраивайтесь поудобнее: в этой статье мы наконец объясним всё так, чтобы понял даже шестилетний гуманитарий!OpenAI (компанию, сделавшую ChatGPT) основали в 2015 году именно вот эти двое парнишек – кто бы тогда знал, во что это в итоге выльется...На всякий случай сразу оговоримся: у этой

In [ ]:
print_highlited("./kristinas_result/text_542718.json")



Обзор современных инструментов дата-аналитика / Habr


              16  February  2021 at 18:16  Обзор современных инструментов дата-аналитика Data Mining *Data visualization *Machine learning *Artificial Intelligence       


Сразу уточню, что видов аналитиков очень много, так как анализировать можно все что угодно. Это и веб-аналитики, и классические data scientists, и бизнес-аналитики, и финансовые аналитики, а также продуктовые, системные и UX аналитики. Причина такого разнообразия, по-видимому, в том, что в ряде крупных компаний над созданием одной платформы или продукта одновременно могут работать десятки, а то и сотни программистов и аналитиков. В таких условиях происходит сильное сужение специализации.


ВСЕ ПЕРЕЧИСЛЕННЫЕ ВИДЫ АНАЛИТИКОВ используют СВОИ специфические наборы инструментов. Поэтому сосредоточусь только непосредственно на сфере анализа данных вне контекста происхождения этих самых данных. Таким образом мы исключаем из обзора системы веб-аналитики, CRM, ERP, сист



# Inlnie semantic tags

In [ ]:
def sort_func(obj):
    return obj["location"]["start"]

In [ ]:
with open("./rucoco/kristina/kristinas_dataset.json", mode="r", encoding="utf8") as kristinas_dataset_file:
    dataset_list = json.load(kristinas_dataset_file, strict=False)
dataset = {}
for data_element in dataset_list:
    dataset[data_element["data_row"]["external_id"]] = data_element["projects"]["clsovprgd0a3x07zq8x8hfvm1"]["labels"][0]["annotations"]["objects"]

In [ ]:
from emoji import UNICODE_EMOJI

def is_emoji(s):
    return s in UNICODE_EMOJI

entries = [entry for entry in os.scandir("./rucoco/kristina/kristinas_texts/texts") if entry.name.endswith(".txt")]
i = 0
for entry in tqdm(entries, leave=True):
    dataset[entry.name].sort(key=sort_func)
    offset = 0
    with open(entry.path, mode="r", encoding="utf8") as f:
        text = f.read().replace("\n", "\\n").replace("\t", "\\t").replace("\r", "\\r").replace("🐺", "🐺🐺")
        for obj in dataset[entry.name]:
            start = obj["location"]["start"]
            end = obj["location"]["end"]
            text = text[:start + offset] + "[" + obj["value"] + " " + text[start + offset:end + offset + 1] + "]" + text[end + offset + 1:]
            label_len = len(obj["value"])
            offset += label_len + 3

        text = text.replace("\\n", "\n").replace("\\t", "\t").replace("\\r", "\r").replace("🐺🐺", "🐺")
        with open(os.path.join("./rucoco/kristina/kristinas_texts/labeled_texts", entry.name), mode="w", encoding="utf8") as labeled_f:
            labeled_f.write(text)
    i += 1
    print("Files: ", i)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 1222.68it/s]

Files:  1
Files:  2
Files:  3
Files:  4
Files:  5
Files:  6
Files:  7
Files:  8
Files:  9
Files:  10
Files:  11
Files:  12
Files:  13
Files:  14
Files:  15
Files:  16
Files:  17
Files:  18
Files:  19
Files:  20
Files:  21


# Split large texts

In [ ]:
import os
from tqdm import tqdm

In [ ]:
tokenizer = load_tokenizer(model.hparams["encoder_model_name"])

In [ ]:
def get_tokens_number(text: str):
    return len(tokenizer.tokenize(text))

In [ ]:
def split_text(text: str, file_name: str, out_dir: str):
    if get_tokens_number(text) <= max_text_length:
        with open(os.path.join(out_dir, file_name), mode="w", encoding="utf8") as f:
            f.write(text)
        return

    start = 0
    end = 0
    texts = []
    while end < len(text):
        new_end = text.find('\n', end + 1)
        if new_end == -1:
            new_end = len(text)
        if get_tokens_number(text[start:new_end]) > max_text_length:
            texts.append(text[start:end])
            start = end + 1
        end = new_end
    texts.append(text[start:end])

    for i in range(len(texts)):
        with open(os.path.join(out_dir,  os.path.splitext(os.path.basename(file_name))[0] + "_" + str(i) + ".txt"), mode="w", encoding="utf8") as f:
               f.write(texts[i])

In [ ]:
entries = [entry for entry in os.scandir("./rucoco/kristina/kristinas_texts/texts") if entry.name.endswith(".txt")]

out_dir = "./rucoco/kristina/kristinas_texts/splitted_texts"
os.makedirs(out_dir, exist_ok=True)

with torch.no_grad():
    i = 0
    for entry in tqdm(entries, leave=True):
        i += 1
        with open(entry.path, mode="r", encoding="utf8") as f:
            text = f.read()
            split_text(text, entry.name, out_dir)

    print("Files: ", i)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:01<00:00, 20.96it/s]

Files:  21


In [ ]:
entries = [entry for entry in os.scandir("./rucoco/kristina/kristinas_texts/texts") if entry.name.endswith(".txt")]


i = 0
for entry in entries:
    i += 1
    with open(entry.path, mode="r", encoding="utf8") as f:
        text = f.read()
        print("Tokens number = " + str(get_tokens_number(text)))


print("Files: ", i)

Tokens number = 2438
Tokens number = 7648
Tokens number = 4096
Tokens number = 4358
Tokens number = 4591
Tokens number = 4575
Tokens number = 14044
Tokens number = 4613
Tokens number = 8824
Tokens number = 3285
Tokens number = 2597
Tokens number = 12238
Tokens number = 2845
Tokens number = 9961
Tokens number = 3583
Tokens number = 3452
Tokens number = 1900
Tokens number = 12544
Tokens number = 3567
Tokens number = 3316
Tokens number = 676
Files:  21


# Predict using data with semantics

Prepare semantics dataset

In [ ]:
import os
from tqdm import tqdm

In [ ]:
tokenizer = load_tokenizer(model.hparams["encoder_model_name"])

NameError: name 'model' is not defined

In [ ]:
with open("./rucoco/kristina/kristinas_dataset.json", mode="r", encoding="utf8") as kristinas_dataset_file:
    dataset_list = json.load(kristinas_dataset_file, strict=False)
dataset0 = {}
for data_element in dataset_list:
    dataset0[data_element["data_row"]["external_id"]] = data_element["projects"]["clsovprgd0a3x07zq8x8hfvm1"]["labels"][0]["annotations"]["objects"]

In [ ]:
def sort_func(obj):
    return obj["location"]["start"]

In [ ]:
entries = [entry for entry in os.scandir("./rucoco/kristina/kristinas_texts/texts") if entry.name.endswith(".txt")]
i = 0
for entry in tqdm(entries, leave=True):
    print(entry.name)
    dataset0[entry.name].sort(key=sort_func)
    with open(entry.path, mode="r", encoding="utf8") as f:
        text = f.read()
        for obj in dataset0[entry.name]:
            start = obj["location"]["start"]
            print("start before shifting: ", obj["location"]["start"])
            offset = text[:start].count("\n") + text[:start].count("\t") + text[:start].count("\r") + text[:start].count("🐺")
            print(offset)
            obj["location"]["start"] -= offset
            obj["location"]["end"] -= offset - 1
            print("start: ", obj["location"]["start"])
            print(text[obj["location"]["start"]:obj["location"]["end"]])
            print("===")
        print("======")
    i += 1

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 386.90it/s]

text_542718.txt
start before shifting:  0
0
start:  0


Обзор с
===
start before shifting:  42
2
start:  40
алитика 
===
start before shifting:  47
2
start:  45
ка / Ha
===
start before shifting:  51
2
start:  49
 Habr


  
===
start before shifting:  63
5
start:  58
  
===
start before shifting:  85
5
start:  80
ar
===
start before shifting:  144
5
start:  139
лити
===
start before shifting:  179
5
start:  174
ion *Machine lear
===
start before shifting:  181
5
start:  176
n *Mac
===
start before shifting:  191
5
start:  186
 learning *Ar
===
start before shifting:  197
5
start:  192
ing *Artificial Intellig
===
start before shifting:  223
5
start:  218
ce       


Сразу
===
start before shifting:  230
8
start:  222
  
===
start before shifting:  244
8
start:  236
уточню, что видов 
===
start before shifting:  250
8
start:  242
, что 
===
start before shifting:  250
8
start:  242
, что видов
===
start before shifting:  277
8
start:  269
ь много
===
start before shifting:  286
8
start:

Split dataset

In [ ]:
import copy

splitted_texts_dir = "./rucoco/kristina/kristinas_texts/splitted_texts"
dataset1 = {}

def get_file_length(file_name: str):
     with open(file_name, mode="r", encoding="utf8") as current_file:
        return len(current_file.read())

for file_name, data in dataset0.items():
    if file_name in os.scandir(splitted_texts_dir):
        dataset1[file_name] = data
        continue

    for obj in data:
        offset = 0
        i = 0
        current_file_name = os.path.splitext(file_name)[0] + "_" + str(i) + ".txt"
        current_file_len = get_file_length(os.path.join(splitted_texts_dir,  current_file_name))

        while obj["location"]["start"] > current_file_len + offset:
            offset += current_file_len + 1
            i += 1
            current_file_name = os.path.splitext(file_name)[0] + "_" + str(i) + ".txt"
            current_file_len = get_file_length(os.path.join(splitted_texts_dir,  current_file_name))

        obj["location"]["start"] -= offset
        obj["location"]["end"] -= offset

        if current_file_name not in dataset1.keys():
            dataset1[current_file_name] = []
        dataset1[current_file_name].append(obj)

In [ ]:
import string

def is_useless(test_str):
    symbols = set(string.whitespace + '\n' + '\r')
    return set(test_str) <= symbols

entries = [entry for entry in os.scandir(splitted_texts_dir) if entry.name.endswith(".txt")]
for entry in entries:
    with open(entry.path, mode="r", encoding="utf-8") as f:
        print("check " + entry.path)
        if is_useless(f.read()):
            os.remove(entry.path)
            print("remove " + entry.path)
            continue
    if entry.name not in dataset1.items():
        dataset1[entry.name] = []

check ./rucoco/kristina/kristinas_texts/splitted_texts/text_753418_2.txt
check ./rucoco/kristina/kristinas_texts/splitted_texts/text_750394_3.txt
check ./rucoco/kristina/kristinas_texts/splitted_texts/text_753418_7.txt
check ./rucoco/kristina/kristinas_texts/splitted_texts/text_758406_0.txt
check ./rucoco/kristina/kristinas_texts/splitted_texts/text_418701_16.txt
check ./rucoco/kristina/kristinas_texts/splitted_texts/text_751972_9.txt
check ./rucoco/kristina/kristinas_texts/splitted_texts/text_747488_7.txt
check ./rucoco/kristina/kristinas_texts/splitted_texts/text_753418_18.txt
check ./rucoco/kristina/kristinas_texts/splitted_texts/text_542718_1.txt
check ./rucoco/kristina/kristinas_texts/splitted_texts/text_751972_2.txt
check ./rucoco/kristina/kristinas_texts/splitted_texts/text_753418_0.txt
check ./rucoco/kristina/kristinas_texts/splitted_texts/text_542718_0.txt
check ./rucoco/kristina/kristinas_texts/splitted_texts/text_751972_4.txt
check ./rucoco/kristina/kristinas_texts/splitted_

In [ ]:
with open(("./rucoco/kristina/dataset_splitted.json"), mode="w", encoding="utf8") as f:
    json.dump(dataset1, f, ensure_ascii=False)

Predict

In [ ]:
model = CorefModel()
model = model.load_from_checkpoint("./rucoco/lightning_logs/version_10_ru_Roberta_large_RuCoCo/checkpoints/last.ckpt")

Some weights of the model checkpoint at ai-forever/ruRoberta-large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ai-forever/ruRoberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be 

In [ ]:
device = "cuda:0"
model.to(device)
model.eval()

CorefModel(
  (encoder): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): Laye

In [ ]:
model.k = 50

In [ ]:
dataset = copy.deepcopy(dataset1)

In [ ]:
logging.disable()

In [ ]:
logging.disable(level=logging.NOTSET)

In [ ]:
out_dir = "./rucoco/kristina/result_RuCoCo_semantics"
os.makedirs(out_dir, exist_ok=True)

entries = [entry for entry in os.scandir("./rucoco/kristina/kristinas_texts/splitted_texts") if entry.name.endswith(".txt")]
with torch.no_grad():
    i = 0
    for entry in tqdm(entries, leave=True):
        i += 1
        with open(entry.path, mode="r", encoding="utf8") as f:
            print(entry.name)
            data = {"text": f.read(), "entities": [], "includes": []}
            doc = Doc(entry.name, data, dataset[entry.name], tokenizer=tokenizer, extract_all_spans=True)
            # doc = Doc(entry.name, data, tokenizer=tokenizer, extract_all_spans=True)
            char_entities = [
                [doc.token_span_to_chars(span) for span in token_entity]
                for token_entity in model.predict(doc)
            ]
            data["entities"] = char_entities
            data["includes"] = [[] for _ in char_entities]
            out_name = os.path.splitext(entry.name)[0] + ".json"
            with open(os.path.join(out_dir, out_name), mode="w", encoding="utf8") as f:
                json.dump(data, f, ensure_ascii=False)
    print("Files: ", i)

  2%|██▊                                                                                                                                        | 3/151 [00:00<00:06, 24.43it/s]WARNING:root:semantic classes: []


text_753418_2.txt
text_750394_3.txt
text_753418_7.txt
text_758406_0.txt
text_418701_16.txt
text_751972_9.txt


  7%|██████████                                                                                                                                | 11/151 [00:00<00:04, 31.44it/s]WARNING:root:semantic classes: []


text_747488_7.txt
text_753418_18.txt
text_542718_1.txt
text_751972_2.txt
text_753418_0.txt
text_542718_0.txt


 13%|█████████████████▎                                                                                                                        | 19/151 [00:00<00:04, 32.71it/s]

...

text_750394_4.txt
text_713920_0.txt
text_713920_4.txt
text_418701_13.txt
text_418701_10.txt
text_713920_2.txt
text_737046_0.txt


 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏ | 149/151 [00:08<00:00, 16.32it/s]WARNING:root:text_732240_0.txt: skipping span (76, 108), 'бизнесе / Habr\n\n\n               '


text_751972_11.txt
text_418701_2.txt
text_753418_13.txt
text_759210_4.txt
text_737046_10.txt
text_732240_0.txt
text_79830_1.txt


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 151/151 [00:08<00:00, 18.62it/s]

Files:  151
